## Build Model from weights files

In [1]:
## To run adversey we need model file not the weight file, so first we have to convert them

from rnn_learner import RNNAgent

# Inferred from weights file
n_actions = 2
state_size = 0
n_cells = 5

#learner_model_path = 'trained_model/RNN_learner_single/gemini/cells_5_gemini/model-49900.weights.h5'
learner_model_path = 'trained_model/RNN_learner_single/cells_5gpt/model-10000.weights.h5'
agent = RNNAgent(n_actions=n_actions, state_size=state_size, n_cells=n_cells, model_path=learner_model_path)
model = agent.model
model.summary()

model.save("trained_model/RNN_learner_single/cells_5gpt/model-10000.weights_final.h5")

2025-07-16 00:14:49,719 - ADV - DEBUG - Weights loaded from trained_model/RNN_learner_single/cells_5gpt/model-10000.weights.h5
2025-07-16 00:14:49,720 - ADV - DEBUG - Model created with 2 actions and 5 cells


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ action_reward (InputLayer)    │ (None, None, 3)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ initial_state (InputLayer)    │ (None, 5)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ GRU (GRU)                     │ (None, None, 5)           │             150 │ action_reward[0][0],       │
│                               │                           │                 │ initial_state[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ policy (Dense)                │ (None, None, 2)           │              12 │ GRU[0][0]                  │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 162 (648.00 B)

 Trainable params: 162 (648.00 B)

 Non-trainable params: 0 (0.00 B)

## Training Adversery

In [2]:
import os
import json
import h5py
import numpy as np
import multiprocessing

from tensorflow.keras.models import load_model, Model
from learner_env import LearnverEnv
from ddqn import DQNAgent
from util.logger import LogFile, DLogger


def load_h5_model_patch_input_layer(path):
    with h5py.File(path, 'r+') as f:
        model_config = f.attrs.get('model_config')
        if model_config is None:
            raise ValueError("No model config found in file.")
        if isinstance(model_config, bytes):
            model_config = model_config.decode('utf-8')
        model_config_json = json.loads(model_config)

        for layer in model_config_json['config']['layers']:
            config = layer['config']
            if 'batch_shape' in config:
                config['batch_input_shape'] = config.pop('batch_shape')

        f.attrs.modify('model_config', json.dumps(model_config_json).encode('utf-8'))

    return load_model(path, compile=False, custom_objects={'Functional': Model})


configs = [{'b': b, 'lr': lr, 'eps': eps}
           for b in [200000, 400000]
           for lr in [0.001, 0.0001, 1e-5]
           for eps in [0.01, 0.1, 0.2]]


def run_adv():
    np.set_printoptions(precision=3)

    buf = configs[9]['b']
    lr = configs[9]['lr']
    eps = configs[9]['eps']

    learner_model_path = 'trained_model/RNN_learner_single/cells_5gpt/model-10000.weights_final.h5'

    # ✅ Construct a clean absolute output path
    base_dir = os.getcwd()
    output_path = os.path.join(base_dir, 'trained_model', f'test_adv_RL_{buf}_eps_{eps}_lr_{lr}', 'gpt')
    os.makedirs(output_path, exist_ok=True)
    print(f'[INFO] Output path: {output_path}')

    with LogFile(output_path, 'run.log'):
        DLogger.logger().debug("Learner model loaded from path {}".format(learner_model_path))
        model = load_h5_model_patch_input_layer(learner_model_path)

        print(model.summary())  # Optional: can be removed if logging is too large

        le = LearnverEnv(model, 2, 1000)
        le.reset()

        # ✅ Create agent and pass correct path
        agent = DQNAgent(
            state_size=le.observation_space.shape[0],
            action_size=4,
            buffer_size=buf,
            epsilon=eps,
            lr=lr
        )

        agent.train(env=le, output_path=output_path, batch_size=1000, total_episodes=int(1e10))


def run(f, n_proc, chunk):
    p = multiprocessing.Pool(n_proc)
    start = min(len(configs), (chunk - 1) * n_proc)
    end = min(len(configs), chunk * n_proc)
    p.map(f, range(start, end))
    p.close()
    p.join()


if __name__ == '__main__':
    run_adv()


[INFO] Output path: C:\Practicum\Practicum_ Adversarial Decision-making of LLMs-20250513T162528Z-1-001\Final_HN\Final_HN\code_H\trained_model\test_adv_RL_400000_eps_0.01_lr_0.001\gpt


2025-07-16 00:17:06,630 - ADV - DEBUG - version control: (None, None)
DEBUG:ADV:version control: (None, None)
2025-07-16 00:17:06,633 - ADV - DEBUG - Learner model loaded from path trained_model/RNN_learner_single/cells_5gpt/model-10000.weights_final.h5
DEBUG:ADV:Learner model loaded from path trained_model/RNN_learner_single/cells_5gpt/model-10000.weights_final.h5


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ action_reward (InputLayer)    │ (None, None, 3)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ initial_state (InputLayer)    │ (None, 5)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ GRU (GRU)                     │ (None, None, 5)           │             150 │ action_reward[0][0],       │
│                               │                           │                 │ initial_state[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ policy (Dense)                │ (None, None, 2)           │              12 │ GRU[0][0]                  │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 162 (648.00 B)

...

dir crearted here -- C:\Practicum\Practicum_ Adversarial Decision-making of LLMs-20250513T162528Z-1-001\Final_HN\Final_HN\code_H\trained_model\test_adv_RL_400000_eps_0.01_lr_0.001\gpt\model-242000.keras


2025-07-16 06:31:15,747 - ADV - DEBUG - episode: 242099/10000000000, score: 0, e: 0.01, reward: 48.328, seudo-rew:0.0 max: 48.86609649658203 init-act: [32.731 49.389 32.734 49.382]
DEBUG:ADV:episode: 242099/10000000000, score: 0, e: 0.01, reward: 48.328, seudo-rew:0.0 max: 48.86609649658203 init-act: [32.731 49.389 32.734 49.382]
2025-07-16 06:31:24,047 - ADV - DEBUG - episode: 242199/10000000000, score: 0, e: 0.01, reward: 48.616, seudo-rew:0.0 max: 48.8846321105957 init-act: [33.314 49.882 33.293 49.466]
DEBUG:ADV:episode: 242199/10000000000, score: 0, e: 0.01, reward: 48.616, seudo-rew:0.0 max: 48.8846321105957 init-act: [33.314 49.882 33.293 49.466]
2025-07-16 06:31:32,231 - ADV - DEBUG - episode: 242299/10000000000, score: 0, e: 0.01, reward: 48.51, seudo-rew:0.0 max: 48.869136810302734 init-act: [33.537 49.688 33.549 49.399]
DEBUG:ADV:episode: 242299/10000000000, score: 0, e: 0.01, reward: 48.51, seudo-rew:0.0 max: 48.869136810302734 init-act: [33.537 49.688 33.549 49.399]
2025-0

dir crearted here -- C:\Practicum\Practicum_ Adversarial Decision-making of LLMs-20250513T162528Z-1-001\Final_HN\Final_HN\code_H\trained_model\test_adv_RL_400000_eps_0.01_lr_0.001\gpt\model-244000.keras


2025-07-16 06:34:00,718 - ADV - DEBUG - episode: 244099/10000000000, score: 0, e: 0.01, reward: 48.065, seudo-rew:0.0 max: 49.439048767089844 init-act: [33.394 50.37  33.605 50.465]
DEBUG:ADV:episode: 244099/10000000000, score: 0, e: 0.01, reward: 48.065, seudo-rew:0.0 max: 49.439048767089844 init-act: [33.394 50.37  33.605 50.465]
2025-07-16 06:34:08,916 - ADV - DEBUG - episode: 244199/10000000000, score: 0, e: 0.01, reward: 48.312, seudo-rew:0.0 max: 49.052635192871094 init-act: [31.906 49.776 31.897 49.585]
DEBUG:ADV:episode: 244199/10000000000, score: 0, e: 0.01, reward: 48.312, seudo-rew:0.0 max: 49.052635192871094 init-act: [31.906 49.776 31.897 49.585]
2025-07-16 06:34:17,157 - ADV - DEBUG - episode: 244299/10000000000, score: 0, e: 0.01, reward: 48.586, seudo-rew:0.0 max: 49.23897933959961 init-act: [32.1   49.775 32.029 49.703]
DEBUG:ADV:episode: 244299/10000000000, score: 0, e: 0.01, reward: 48.586, seudo-rew:0.0 max: 49.23897933959961 init-act: [32.1   49.775 32.029 49.703]


KeyboardInterrupt: 

In [1]:
import os
import json
import h5py
import numpy as np
from tensorflow.keras.models import load_model, Model
from learner_env import LearnverEnv
from ddqn import DQNAgent
from util.logger import LogFile, DLogger

# Utility: fix Keras HDF5 compatibility
def load_h5_model_patch_input_layer(path):
    with h5py.File(path, 'r+') as f:
        model_config = f.attrs.get('model_config')
        if model_config is None:
            raise ValueError("No model config found in file.")
        if isinstance(model_config, bytes):
            model_config = model_config.decode('utf-8')
        model_config_json = json.loads(model_config)
        for layer in model_config_json['config']['layers']:
            config = layer['config']
            if 'batch_shape' in config:
                config['batch_input_shape'] = config.pop('batch_shape')
        f.attrs.modify('model_config', json.dumps(model_config_json).encode('utf-8'))
    return load_model(path, compile=False, custom_objects={'Functional': Model})

# Training function for one config
def train_adv_model(buf, lr, eps, learner_name="gpt", total_episodes=50000):
    np.set_printoptions(precision=3)

    # Path to learner model (RNN policy)
    learner_model_path = f'trained_model/RNN_learner_single/cells_5{learner_name}/model-49900.weights_final.h5'
    if not os.path.exists(learner_model_path):
        print(f"⚠ Learner model not found: {learner_model_path}")
        return

    # Output path
    output_path = os.path.join(
        'trained_model',
        f'test_adv_RL_{buf}eps{eps}lr{lr}',
        learner_name
    )
    os.makedirs(output_path, exist_ok=True)
    print(f'📦 Training: buf={buf}, lr={lr}, eps={eps} → {output_path}')

    with LogFile(output_path, 'run.log'):
        DLogger.logger().debug("Loading learner model from {}".format(learner_model_path))
        model = load_h5_model_patch_input_layer(learner_model_path)

        env = LearnverEnv(model, 2, 1000)
        env.reset()

        agent = DQNAgent(
            state_size=env.observation_space.shape[0],
            action_size=4,
            buffer_size=buf,
            epsilon=eps,
            lr=lr
        )

        agent.train(
            env=env,
            output_path=output_path,
            batch_size=1000,
            total_episodes=total_episodes
        )

# ------------------------------
# 🔧 Hyperparameter Grid
# ------------------------------
buffers = [400000]
learning_rates = [0.001, 0.0001, 1e-5]
epsilons = [0.01, 0.1, 0.2]

# Choose learner: 'gpt', 'gemini', etc.
learner_name = "gpt"  # or "gemini"
episodes_per_run = 50000  # reduced for tuning

# Run grid search
for buf in buffers:
    for lr in learning_rates:
        for eps in epsilons:
            print(buf, lr, eps)
            train_adv_model(buf, lr, eps, learner_name, episodes_per_run)

400000 0.001 0.01
📦 Training: buf=400000, lr=0.001, eps=0.01 → trained_model\test_adv_RL_400000eps0.01lr0.001\gpt


2025-07-17 23:15:15,819 - ADV - DEBUG - version control: (None, None)
2025-07-17 23:15:15,822 - ADV - DEBUG - Loading learner model from trained_model/RNN_learner_single/cells_5gpt/model-49900.weights_final.h5
C:\Users\nitis\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-07-17 23:15:16,447 - ADV - DEBUG - Trained model saved to: trained_model\test_adv_RL_400000eps0.01lr0.001\gpt\model-0.keras


dir crearted here -- trained_model\test_adv_RL_400000eps0.01lr0.001\gpt\model-0.keras


2025-07-17 23:15:23,694 - ADV - DEBUG - episode: 99/50000, score: 0, e: 0.01, reward: 19.884, seudo-rew:0.0 max: 0.9324346780776978 init-act: [-0.281  0.32   1.233  0.153]
2025-07-17 23:15:32,250 - ADV - DEBUG - episode: 199/50000, score: 0, e: 0.01, reward: 20.012, seudo-rew:0.0 max: 1.0522441864013672 init-act: [0.379 0.828 1.361 0.768]
2025-07-17 23:15:39,891 - ADV - DEBUG - episode: 299/50000, score: 0, e: 0.01, reward: 21.851, seudo-rew:0.0 max: 1.3724719285964966 init-act: [0.97  1.402 1.531 1.542]
2025-07-17 23:15:47,143 - ADV - DEBUG - episode: 399/50000, score: 0, e: 0.01, reward: 28.907, seudo-rew:0.0 max: 2.0750691890716553 init-act: [1.658 1.973 1.76  2.936]
2025-07-17 23:15:58,134 - ADV - DEBUG - episode: 499/50000, score: 0, e: 0.01, reward: 35.852, seudo-rew:0.0 max: 3.334939479827881 init-act: [2.69  3.061 2.007 3.932]
2025-07-17 23:16:08,564 - ADV - DEBUG - episode: 599/50000, score: 0, e: 0.01, reward: 35.249, seudo-rew:0.0 max: 4.302211284637451 init-act: [3.508 3.98

dir crearted here -- trained_model\test_adv_RL_400000eps0.01lr0.001\gpt\model-2000.keras


...

2025-07-18 09:41:22,444 - ADV - DEBUG - episode: 44099/50000, score: 0, e: 0.2, reward: 27.636, seudo-rew:0.0 max: 40.45881652832031 init-act: [37.174 41.281 37.348 41.258]
2025-07-18 09:41:30,857 - ADV - DEBUG - episode: 44199/50000, score: 0, e: 0.2, reward: 27.908, seudo-rew:0.0 max: 40.32008743286133 init-act: [37.04  41.149 37.19  41.086]
2025-07-18 09:41:39,156 - ADV - DEBUG - episode: 44299/50000, score: 0, e: 0.2, reward: 27.672, seudo-rew:0.0 max: 40.211936950683594 init-act: [36.891 41.017 37.041 40.921]
2025-07-18 09:41:47,483 - ADV - DEBUG - episode: 44399/50000, score: 0, e: 0.2, reward: 27.672, seudo-rew:0.0 max: 40.1016960144043 init-act: [36.738 40.83  36.875 40.779]
2025-07-18 09:41:55,754 - ADV - DEBUG - episode: 44499/50000, score: 0, e: 0.2, reward: 27.958, seudo-rew:0.0 max: 39.94548416137695 init-act: [36.606 40.734 36.754 40.699]
2025-07-18 09:42:04,110 - ADV - DEBUG - episode: 44599/50000, score: 0, e: 0.2, reward: 27.925, seudo-rew:0.0 max: 39.81277084350586 in

dir crearted here -- trained_model\test_adv_RL_400000eps0.2lr1e-05\gpt\model-46000.keras


2025-07-18 09:44:09,734 - ADV - DEBUG - episode: 46099/50000, score: 0, e: 0.2, reward: 27.887, seudo-rew:0.0 max: 37.31062698364258 init-act: [34.164 38.17  34.293 38.055]
2025-07-18 09:44:18,004 - ADV - DEBUG - episode: 46199/50000, score: 0, e: 0.2, reward: 27.863, seudo-rew:0.0 max: 37.10636520385742 init-act: [34.042 38.016 34.168 37.914]
2025-07-18 09:44:26,311 - ADV - DEBUG - episode: 46299/50000, score: 0, e: 0.2, reward: 27.979, seudo-rew:0.0 max: 36.93326950073242 init-act: [33.857 37.773 34.004 37.69 ]
2025-07-18 09:44:34,659 - ADV - DEBUG - episode: 46399/50000, score: 0, e: 0.2, reward: 27.813, seudo-rew:0.0 max: 36.7044563293457 init-act: [33.67  37.544 33.824 37.502]
2025-07-18 09:44:42,926 - ADV - DEBUG - episode: 46499/50000, score: 0, e: 0.2, reward: 28.08, seudo-rew:0.0 max: 36.49033737182617 init-act: [33.521 37.349 33.672 37.307]
2025-07-18 09:44:51,171 - ADV - DEBUG - episode: 46599/50000, score: 0, e: 0.2, reward: 28.048, seudo-rew:0.0 max: 36.32480239868164 init

dir crearted here -- trained_model\test_adv_RL_400000eps0.2lr1e-05\gpt\model-48000.keras


2025-07-18 09:46:56,669 - ADV - DEBUG - episode: 48099/50000, score: 0, e: 0.2, reward: 28.673, seudo-rew:0.0 max: 34.2121467590332 init-act: [31.432 35.021 31.519 34.961]
2025-07-18 09:47:05,070 - ADV - DEBUG - episode: 48199/50000, score: 0, e: 0.2, reward: 28.77, seudo-rew:0.0 max: 34.070152282714844 init-act: [31.301 34.894 31.389 34.798]
2025-07-18 09:47:13,400 - ADV - DEBUG - episode: 48299/50000, score: 0, e: 0.2, reward: 28.746, seudo-rew:0.0 max: 33.979305267333984 init-act: [31.211 34.805 31.266 34.717]
2025-07-18 09:47:21,632 - ADV - DEBUG - episode: 48399/50000, score: 0, e: 0.2, reward: 28.956, seudo-rew:0.0 max: 33.83966827392578 init-act: [31.084 34.652 31.128 34.582]
2025-07-18 09:47:29,996 - ADV - DEBUG - episode: 48499/50000, score: 0, e: 0.2, reward: 28.674, seudo-rew:0.0 max: 33.70817184448242 init-act: [30.984 34.573 31.047 34.488]
2025-07-18 09:47:38,308 - ADV - DEBUG - episode: 48599/50000, score: 0, e: 0.2, reward: 28.809, seudo-rew:0.0 max: 33.619266510009766 i

## Keras to H5

In [1]:
import os
from tensorflow.keras.models import load_model

def convert_keras_to_h5(keras_model_path, h5_output_path=None):
    """
    Converts a .keras model file to .h5 format.

    Args:
        keras_model_path (str): Path to the input .keras file.
        h5_output_path (str): Output path for the .h5 file.
    """
    if not keras_model_path.endswith('.keras'):
        raise ValueError("Input file must be a '.keras' file")

    if h5_output_path is None:
        h5_output_path = keras_model_path.replace('.keras', '.h5')

    # Load and save
    model = load_model(keras_model_path, compile=False)
    model.save(h5_output_path, save_format='h5')
    print(f"[✓] Model converted and saved to: {h5_output_path}")

# === Example Usage ===
if __name__ == '__main__':
    input_keras_path = 'trained_model/adv_RL_400000_eps_0.01_lr_0.001/gpt/model-248000.keras'
    output_h5_path = 'trained_model/adv_RL_400000_eps_0.01_lr_0.001/gpt/model-248000_final.h5'

    os.makedirs(os.path.dirname(output_h5_path), exist_ok=True)
    convert_keras_to_h5(input_keras_path, output_h5_path)


[✓] Model converted and saved to: trained_model/adv_RL_400000_eps_0.01_lr_0.001/gpt/model-248000_final.h5
